In [10]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [11]:
df = pd.read_csv('data/merged_df.csv')

In [12]:
df_lgbm = df.copy()
#Scaling numerical columns
scaler = StandardScaler()
df_lgbm[df_lgbm._get_numeric_data().columns] = scaler.fit_transform(df_lgbm[df_lgbm._get_numeric_data().columns])  


for c in df_lgbm.columns:
    col_type = df_lgbm[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df_lgbm[c] = df_lgbm[c].astype('category')

x_lgbm = df_lgbm.drop(['_SALE_PRICE_'],axis=1)
y_lgbm=df_lgbm['_SALE_PRICE_'].astype('int64')

x_train, x_test, y_train, y_test = train_test_split(x_lgbm, y_lgbm, test_size=0.20, random_state=314)

In [18]:
#Training lgbm

lgb_train = lgb.Dataset(x_train,y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mse'},
    'num_leaves': 10,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=50)

[1]	valid_0's l2: 0.353735
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.34275
[3]	valid_0's l2: 0.333135
[4]	valid_0's l2: 0.325506
[5]	valid_0's l2: 0.314234
[6]	valid_0's l2: 0.308454
[7]	valid_0's l2: 0.302212
[8]	valid_0's l2: 0.297737
[9]	valid_0's l2: 0.294335
[10]	valid_0's l2: 0.291734
[11]	valid_0's l2: 0.289214
[12]	valid_0's l2: 0.287503
[13]	valid_0's l2: 0.285233
[14]	valid_0's l2: 0.283922
[15]	valid_0's l2: 0.278911
[16]	valid_0's l2: 0.277633
[17]	valid_0's l2: 0.277023
[18]	valid_0's l2: 0.274567
[19]	valid_0's l2: 0.272599
[20]	valid_0's l2: 0.270399
[21]	valid_0's l2: 0.267639
[22]	valid_0's l2: 0.265354
[23]	valid_0's l2: 0.264339
[24]	valid_0's l2: 0.260994
[25]	valid_0's l2: 0.260399
[26]	valid_0's l2: 0.259044
[27]	valid_0's l2: 0.259355
[28]	valid_0's l2: 0.258856
[29]	valid_0's l2: 0.261306
[30]	valid_0's l2: 0.260185
[31]	valid_0's l2: 0.259912
[32]	valid_0's l2: 0.259839
[33]	valid_0's l2: 0.259951
[34]	valid_0's l2: 0.258

In [19]:
le = preprocessing.LabelEncoder()

for c in df.columns:
    col_type = df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df[c] = le.fit_transform(df[c])

df[df._get_numeric_data().columns] = scaler.fit_transform(df[df._get_numeric_data().columns])  

x = df.drop(['_SALE_PRICE_'],axis=1)
y=df['_SALE_PRICE_'].astype('int64')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=314)

clf = RandomForestClassifier(n_estimators=1000, max_depth=10,random_state=0)
clf.fit(x, y)

#testing the model
predictions = clf.predict(x_test)
mean_squared_error(y_test,predictions)


0.36482621132010057